In [1]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import IPython

sys.path.append("../src/utils")
sys.path.append("../src")

In [2]:
%load_ext autoreload
%autoreload 1
%aimport datasets
%aimport general
%aimport training
%aimport generation

In [3]:
from general import *

In [50]:
facebook_paths = ['../data/social/facebook2/']

def load_all(loader, dir):
    buff = []
    if not os.path.isdir(dir): return buff
    for i in os.listdir(dir):
        buff.append(loader(dir + i))
    return buff
        

chats = []

for fb in facebook_paths:
    chats += load_all(datasets.load_messenger, fb + 'messages/inbox/')
    
original_chats = chats

def to_template(chat):
    buff = []
    for msg in chat:
        buff.append({'user': msg.user,
                     'time': msg.time,
                     'content': msg.body})
    return buff

template_chats = lmap(to_template, original_chats)

In [53]:
from transformers import GPT2Tokenizer, AutoTokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_tokens([tok for tok in datasets.CONTROL_TOKENS.values() if tok != 'eos'])

chat_template = open("../src/utils/chat_template.jinja").read()
tokenizer.chat_template = chat_template
tokenizer.apply_chat_template(template_chats[0], tokenize=False)[:500]

"Vojta Polena<|goes|>Hej\nPatztablook TwentyTwo<|goes|>cheers\nPatztablook TwentyTwo<|goes|>You can now call each other and see information such as Active Status and when you've read messages.\nVojta Polena<|goes|>Chtěl jsi tady udělat nějakou skupinu?\nPatztablook TwentyTwo<|goes|>jj\nPatztablook TwentyTwo<|goes|>večer si k tomu sednu\nPatztablook TwentyTwo<|goes|>+ ještě čekám na víťu\nVojta Polena<|goes|>okok\nVojta Polena<|goes|>Hele asi bych tu konverzaci okolo toho výletu přesunul do pm\nVojta Polen"

In [5]:
actions_path = '../data/actions.pkl'

import pickle
with open(actions_path, 'rb') as f:
    actions = pickle.load(f)

In [6]:
untokenized = lmap(datasets.action_to_string, lfilter(lambda a: a.user == 'p', actions[0]))
model = generation.SamplingModel(tokenizer=tokenizer, untokenized=untokenized)

In [7]:
model.generate([[1,2,3]])

[[1, 2, 3, 79, 50257, 50260, 50258]]

In [66]:
pipeline = generation.Pipeline(model=model, tokenizer=tokenizer)

for action in pipeline([datasets.Message(user='juzr', time=0, body='hello there')]):
    print(action)

Action(time=0, user='', type='message', data={'followup': True, 'body': 'p'})
Action(time=0, user='', type='message', data={'followup': True, 'body': 'p <|goes|>'})
Action(time=0, user='', type='message', data={'followup': True, 'body': 'p <|goes|> <|typing|>'})
Action(time=0, user='', type='message', data={'followup': True, 'body': 'p <|goes|> <|typing|>'})
Action(time=0, user='', type='idle', data={'followup': True})
